# ARIMA

ARIMA (Auto-Regressive Integrated Moving Average)
-The biggest advantage of this model is that it can be applied in case where the data shows evidence of non-stationarity. 

A quick summary of ARIMA
Arims model for Time Series Forcasting
Arima is specified by three order parameters: (p, d, q).

AR(p) Autoregression - An auto regressice (AR(p)) component refers to the use of past values in the regression equation for the time series.
uppercase P may be used depending on seasonality. 
p component controlls/defines the autoregression part

I(d) Integration- uses differencing of observations (subtracting an observation from observation at the previous time step) in order to make the time series stationary.

MA(q) Moving Average-A moving average component depicts the error of the model as a combination of previous error terms. The order q represents the number of terms to be included in the model. 

Various flavors of ARIMA Model:
ARIMA: Non-seasonal Autoregressive Integrated Moving Averages
SARIMA: Seasonal ARIMA
SARIMAX: Seasonal ARIMA with exogeneous variables. 

Pyramid Auto_ARIMA
The auto_arima function from the pmdarima library helps identify the most optimal parameters for an ARIMA model and returns a fitted ARIMA model. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
df=pd.read_csv('df_train_callvolume (1).csv')

In [3]:
df.dtypes

INTERVAL             object
Account               int64
AccountClose          int64
AccountOnline         int64
Account_Issue         int64
ApplicationStatus     int64
BillingQuery          int64
CardDeclined          int64
Card_Compromise       int64
ChargeReview          int64
Complaint             int64
DataBreach            int64
Entertainment         int64
FraudQuery            int64
Online_Login          int64
PaymentExecute        int64
Payments              int64
Representative        int64
Rewards               int64
Statement             int64
Travel                int64
dtype: object

In [4]:
df['INTERVAL']=pd.to_datetime(df['INTERVAL'])
df.dtypes

INTERVAL             datetime64[ns]
Account                       int64
AccountClose                  int64
AccountOnline                 int64
Account_Issue                 int64
ApplicationStatus             int64
BillingQuery                  int64
CardDeclined                  int64
Card_Compromise               int64
ChargeReview                  int64
Complaint                     int64
DataBreach                    int64
Entertainment                 int64
FraudQuery                    int64
Online_Login                  int64
PaymentExecute                int64
Payments                      int64
Representative                int64
Rewards                       int64
Statement                     int64
Travel                        int64
dtype: object

In [5]:
df.set_index('INTERVAL', inplace=True)

In [6]:
df.head()

,Account,AccountClose,AccountOnline,Account_Issue,ApplicationStatus,BillingQuery,CardDeclined,Card_Compromise,ChargeReview,Complaint,DataBreach,Entertainment,FraudQuery,Online_Login,PaymentExecute,Payments,Representative,Rewards,Statement,Travel
INTERVAL,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,5,27,8,2,3,17,15,0,37,0,0,0,21,13,97,22,165,19,1,19
2018-01-01 00:15:00,2,15,5,1,0,8,18,2,30,1,0,0,25,2,93,19,138,23,5,20
2018-01-01 00:30:00,2,20,7,0,1,4,16,1,28,1,0,0,13,9,87,22,166,14,2,27
2018-01-01 00:45:00,1,22,3,0,2,10,13,0,31,0,0,0,20,7,79,26,155,12,2,33
2018-01-01 01:00:00,1,18,4,0,1,6,17,2,26,0,0,0,25,4,94,14,136,16,2,25


## EDA

Is the data stationary?
Dickey-Fuller test
if pvalue is above 0.05, fail to reject null hypothesis and data is non-stationary.

In [7]:
from statsmodels.tsa.stattools import adfuller
for coloumn in df:
    print(coloumn)
    adfuller_test = adfuller(df[coloumn], autolag= "AIC")
    print("ADF test statistic: {}".format(adfuller_test[0]))
    print("p-value: {}".format(adfuller_test[1]))

Account
ADF test statistic: -28.20241222960798
p-value: 0.0
AccountClose
ADF test statistic: -28.173617567613057
p-value: 0.0
AccountOnline
ADF test statistic: -43.273662784421056
p-value: 0.0
Account_Issue
ADF test statistic: -40.24677309018842
p-value: 0.0
ApplicationStatus
ADF test statistic: -37.89816678145959
p-value: 0.0
BillingQuery
ADF test statistic: -22.73963642381993
p-value: 0.0
CardDeclined
ADF test statistic: -43.35062181411422
p-value: 0.0
Card_Compromise
ADF test statistic: -25.954431543829102
p-value: 0.0
ChargeReview
ADF test statistic: -34.22446515835775
p-value: 0.0
Complaint
ADF test statistic: -38.62068640399181
p-value: 0.0
DataBreach
ADF test statistic: -19.043498588688916
p-value: 0.0
Entertainment
ADF test statistic: -26.54447256866831
p-value: 0.0
FraudQuery
ADF test statistic: -38.01440791330454
p-value: 0.0
Online_Login
ADF test statistic: -43.122205374040576
p-value: 0.0
PaymentExecute
ADF test statistic: -27.466259689183865
p-value: 0.0
Payments
ADF test 

In [8]:
from pmdarima.arima import ADFTest
for coloumn in df:
    #print(coloumn)
    adf_test = ADFTest(alpha=0.05)
    adf_test.should_diff(df[coloumn])

In [9]:
adf_test = ADFTest(alpha=0.05)
adf_test.should_diff(df['Account'])

(0.01, False)

In [10]:
df_test=df['Account'].iloc[:672]

In [11]:
df_test.head()

INTERVAL
2018-01-01 00:00:00    5
2018-01-01 00:15:00    2
2018-01-01 00:30:00    2
2018-01-01 00:45:00    1
2018-01-01 01:00:00    1
Name: Account, dtype: int64

In [12]:
adfuller_test = adfuller(df_test, autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

ADF test statistic: -4.494677001974115
p-value: 0.00020104098323942763


In [13]:
df_test2=df['Account'].iloc[:2688]

In [14]:
df_test2.head()

INTERVAL
2018-01-01 00:00:00    5
2018-01-01 00:15:00    2
2018-01-01 00:30:00    2
2018-01-01 00:45:00    1
2018-01-01 01:00:00    1
Name: Account, dtype: int64

In [15]:
adfuller_test = adfuller(df_test2, autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

ADF test statistic: -12.76047166577165
p-value: 8.164215610663465e-24


Use Johansen’s test for checking the stationarity of any multivariate time series data

In [16]:
df1=df.loc[:,"Account":"Entertainment"]

In [17]:
df1.head()

,Account,AccountClose,AccountOnline,Account_Issue,ApplicationStatus,BillingQuery,CardDeclined,Card_Compromise,ChargeReview,Complaint,DataBreach,Entertainment
INTERVAL,,,,,,,,,,,,
2018-01-01 00:00:00,5,27,8,2,3,17,15,0,37,0,0,0
2018-01-01 00:15:00,2,15,5,1,0,8,18,2,30,1,0,0
2018-01-01 00:30:00,2,20,7,0,1,4,16,1,28,1,0,0
2018-01-01 00:45:00,1,22,3,0,2,10,13,0,31,0,0,0
2018-01-01 01:00:00,1,18,4,0,1,6,17,2,26,0,0,0


In [18]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
coint_johansen(df1,-1,1).eig

array([0.32352274, 0.31703438, 0.31491514, 0.3053471 , 0.29340183,
       0.1982739 , 0.17423272, 0.10516682, 0.05726595, 0.04115767,
       0.03257633, 0.0023558 ])

In [19]:
df2=df.loc[:,"FraudQuery":]

In [20]:
df2.head()

,FraudQuery,Online_Login,PaymentExecute,Payments,Representative,Rewards,Statement,Travel
INTERVAL,,,,,,,,
2018-01-01 00:00:00,21,13,97,22,165,19,1,19
2018-01-01 00:15:00,25,2,93,19,138,23,5,20
2018-01-01 00:30:00,13,9,87,22,166,14,2,27
2018-01-01 00:45:00,20,7,79,26,155,12,2,33
2018-01-01 01:00:00,25,4,94,14,136,16,2,25


In [21]:
coint_johansen(df2,-1,1).eig

array([0.27085583, 0.2468065 , 0.21756936, 0.1551461 , 0.0973913 ,
       0.05594207, 0.04132155, 0.00290971])

## ARIMA Modeling

Autoarima gives us bet model suited for the data
p - number of autoregressive terms (AR)
q - Number of moving avergae terms (MA)
d - number of non-seasonal differences
p, d, q represent non-seasonal components
P, D, Q represent seasonal components

In [22]:
from pmdarima.arima import auto_arima

In [23]:
arima_model = auto_arima(df['Account'], start_p = 1, d=1, start_q = 1, 
                          max_p = 5, max_q = 5, max_d=5,
                         
                          start_P = 0, D=1, start_Q=0, max_P=5, max_D=5, max_Q=5,
                          seasonal = True, 
                          trace = True, 
                          error_action ='ignore',   
                          suppress_warnings = True,  
                          stepwise = True, n_fits=7) 

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=173432.338, Time=4.41 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=186974.332, Time=0.70 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=177755.720, Time=0.82 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=173601.918, Time=2.58 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=186972.332, Time=0.36 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=173333.217, Time=6.28 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=174830.983, Time=1.76 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=173229.457, Time=10.03 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=173599.734, Time=2.19 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=173188.438, Time=10.76 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=173222.311, Time=2.56 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=173183.375, Time=19.66 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=173181.601, Time=3.08 sec
 ARIMA(5,1,0)(0,0,0)[0]             : AIC=173179.601, Time=1.73 sec
 A

In [24]:
arima_model = auto_arima(df['Account'], trace = True, error_action ='ignore',   
                          suppress_warnings = True)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=172367.622, Time=17.00 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=186974.332, Time=0.67 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=177755.720, Time=0.84 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=173601.918, Time=2.56 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=186972.332, Time=0.38 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=172376.761, Time=15.57 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=173333.217, Time=6.14 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=172357.167, Time=26.39 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=173229.457, Time=9.88 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=172357.879, Time=38.11 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=172367.812, Time=16.58 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=172380.402, Time=20.79 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=173188.438, Time=10.74 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=172360.122, Time=27.86 s

In [25]:
# To print the summary 
print(arima_model.summary() ) #Note down the Model and details.

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                35040
Model:               SARIMAX(3, 1, 2)   Log Likelihood              -86171.583
Date:                Fri, 19 Nov 2021   AIC                         172355.167
Time:                        15:16:42   BIC                         172405.952
Sample:                             0   HQIC                        172371.342
                              - 35040                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9504      0.007    133.246      0.000       0.936       0.964
ar.L2         -0.0166      0.006     -2.718      0.007      -0.029      -0.005
ar.L3         -0.0258      0.006     -4.378      0.0

m->seasonal cycle length
    define frequency as day= 96
                        week=672

max_order -> determines the total number of non seasonal AR and MA components the model can have. aka max value for sum of p and q. setting it equal to none makes the auto_arima more flexible b/c it can try any combination of values but increases the computational time significantly so were going to ignore this for now. 

max_p, max_q, max_d-> Maximum AR, MA, integration component respectively default is 5 

maxiter-> defualt is 50

In [ ]:
arima_model = auto_arima(df['Account'], start_p = 1, d=1, start_q = 1, 
                          max_p = 5, max_q = 5, max_d=5, m =96, 
                          start_P = 0, D=1, start_Q=0, max_P=5, max_D=5, max_Q=5,
                          seasonal = True, 
                          trace = True, 
                          error_action ='ignore',   
                          suppress_warnings = True,  
                          stepwise = True, n_fits=7) 

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,0)[96]             : AIC=191639.986, Time=455.91 sec
